In [ ]:
import numpy as np
import copy
import plotly.graph_objects as go

# Clases importantes
Estas clases permiten construir curva nurbs dados los puntos de control, nodos y pesos.  
La clase Construccion construye circunferencias Nurbs a partir de poligonos, importante ya que la curva directriz de estas superficies no orientabes es una circunferencia.

In [ ]:
class Bases_curves:
    def __init__(self, t=None):
        if t is None:
            t = [0, 1, 2, 3, 4]
        self.t = t

    def N_0(self, i, u):
        if i <= len(self.t) - 2:
            return (lambda u_val: 1 if ((u_val >= self.t[i]) & (u_val < self.t[i + 1])) else 0)(u)
        else:
            return 'El índice ha sobrepasado los límites'

    def N_p(self, i, u, p):
        if (p <= len(self.t) - 2) & (i <= len(self.t) - p - 2):
            if p == 0:
                return self.N_0(i, u)
            else:
                return ((lambda u_val: ((u_val - self.t[i]) / (self.t[i + p] - self.t[i]) * self.N_p(i, u, p - 1) 
                        if self.t[i + p] - self.t[i] != 0 else 0))(u) + 
                        (lambda u_val: ((self.t[i + p + 1] - u_val) / (self.t[i + p + 1] - self.t[i + 1]) * self.N_p(i + 1, u, p - 1) 
                        if self.t[i + p + 1] - self.t[i + 1] != 0 else 0))(u))
        else:
            return f'Ingrese valores de i=[0, {len(self.t) - p - 2}] y p=[0, {len(self.t) - 2}]'

In [ ]:
class Nurbs_curbs():
    def __init__(self):
        pass

    def curva(self, pts, pes, nod, p, u):
        suma_den, suma_num = 0, [0 for k in range(len(pts[0]))]
        for i in range(len(pes)):
            suma_den = suma_den + (lambda i_val: (pes[i_val]*Bases_curves(nod).N_p(i=i_val, u=u, p=p)))(i)

            suma_num = [a + b for a,b in zip(suma_num, (lambda i_val: ([
                pes[i_val]*Bases_curves(nod).N_p(i=i_val, u=u, p=p)*psdr for psdr in pts[i_val]]))(i)) ]
        
        return [psdr/suma_den for psdr in suma_num]

    def curva1(self, pts, pes, nod, p, u):
        if isinstance(u, (int, float)):  
            instancia = self.curva(pts, pes, nod, p, u)
            return instancia
        elif isinstance(u, (list, np.ndarray)):  
            instancias = [self.curva(pts, pes, nod, p, num) for num in u]
            return instancias
        else:
            raise ValueError("La entrada debe ser un número o un array de NumPy.")

In [ ]:
class Construccion():
    def __init__(self, n, r):
        self.n, self.r = n, r

    def polygon(self):
        θ = 2*np.pi/self.n
        α = np.pi*(self.n-2)/(2*self.n)
        R = self.r/np.sin(α)
        Tabla, Lista=[[[self.r*np.cos(θ*k),self.r*np.sin(θ*k)],[R*np.cos(θ*(2*k+1)/2),R*np.sin(θ*(2*k+1)/2)]] for k in range(self.n)], []
        for cont in range(len(Tabla)):
            Lista.extend([Tabla[cont][0], Tabla[cont][1]])
        Lista.append(Tabla[0][0])
        return Lista

    def peso(self):
        θ = 2*np.pi/self.n
        Tabla, Lista = [[1, np.cos(θ/2)] for k in range(self.n)], []
        for cont in range(len(Tabla)):
            Lista.extend(Tabla[cont])
        Lista.append(Tabla[0][0])
        return Lista

    def open1(self):
        Tabla, Lista = [[k,k] for k in range(self.n+1)], []
        for cont in range(len(Tabla)):
            Lista.extend(Tabla[cont])
        Lista = [0] + Lista + [self.n]
        return Lista

# Clase para graficar superficies rotacion y giro
Aqui pts1 esta en el **plano xy** y es la curva de **rotación**, es decir, **directriz**.
pts3 esta en el **plano xz** y es la curva de giro (semicircunferencia).
por último, pts2 esta en el **plano xz** y es la curva **generatriz**. Además se tiene **direction** que es la curva en el plano **xy** que es por donde pasara la superficie. Cade señalar que  pts1, pts3 y direction deben tener mismo tamaño en números de puntos de control.

In [ ]:
class Nurbs_surface():
    def __init__(self, pts1, pts2, pts3, direction, pes1, pes2, nod1, nod2, p1, p2):
        self.pts1, self.pts2, self.pts3, self.pes1, self.pes2, self.nod1, self.nod2, self.p1, self.p2 = pts1, pts2, pts3, pes1, pes2, nod1, nod2, p1, p2
        self.direction = direction
        self.objeto1, self.objeto2 = Bases_curves(t=self.nod1), Bases_curves(t=self.nod2)

    def W_ij(self, i, j):
        w_ij = lambda i_val, j_val: (self.pes1[i_val]*self.pes2[j_val] if (i_val < len(self.pes1) and j_val < len(self.pes2))
                                       else 'índices sobrepasaron límites')
        return w_ij(i, j)

    def P_ij(self, i, j):
        p_ij = lambda i_val, j_val: ([self.direction[i_val][0]+self.pts2[j_val][0]*self.pts1[i_val][0]*self.pts3[i_val][0]-self.pts2[j_val][2]*self.pts1[i_val][0]*self.pts3[i_val][2],
                                      self.direction[i_val][1]+self.pts2[j_val][0]*self.pts1[i_val][1]*self.pts3[i_val][0]-self.pts2[j_val][2]*self.pts1[i_val][1]*self.pts3[i_val][2],
                                      self.pts2[j_val][0]*self.pts3[i_val][2]+self.pts2[j_val][2]*self.pts3[i_val][0]] 
                                     if (i_val < len(self.pts1) and j_val < len(self.pts2)) else 'índices sobrepasaron límites')
        return p_ij(i,j)
    
    def superficie(self, u, v):
        suma_den, suma_num = 0, [0 for k in range(len(self.pts1[0]))]
        for i1 in range(len(self.pes1)):
            for j1 in range(len(self.pes2)):
                suma_den = suma_den + (lambda i_val, j_val: (
                    self.objeto1.N_p(i=i_val, u=u, p=self.p1)*self.objeto2.N_p(i=j_val, u=v, p=self.p2)*self.W_ij(i_val, j_val)))(i1, j1)

                suma_num = [a + b for a,b in zip(suma_num, (lambda i_val, j_val: ([
                    self.objeto1.N_p(i=i_val, u=u, p=self.p1)*self.objeto2.N_p(i=j_val, u=v, p=self.p2)*self.W_ij(i_val, j_val)*psdr
                    for psdr in self.P_ij(i_val, j_val)]))(i1,j1)) ]
        return [psdr/suma_den for psdr in suma_num]

    def procesar(self, u, v):
        if (isinstance(u, (int, float))) & (isinstance(v, (int, float))):
            instancia = self.superficie(u, v)
            return instancia
        elif (isinstance(u, (np.ndarray))) & (isinstance(v, (np.ndarray))):
            if u.shape != v.shape:
                return 'Ingrese vectores con mismas dimensiones'
            else:
                fila, columna = u.shape
                lista_alm = [[] for k in range( len(self.superficie(u[0][0], v[0][0]))  )]
                for i in range(fila):
                    for j in range(columna):
                        B = self.superficie(u[i][j], v[i][j])
                        for k in range(len(B)):
                            lista_alm[k].append(B[k])
                C = [np.array(psdr).reshape(fila, columna) for psdr in lista_alm]
                return C
        else:
            raise ValueError("La entrada debe ser un número o un array de NumPy.")

In [ ]:
class Nurbs_NonOrientPy_graph:
    def __init__(self, pts1, pts2, pts3, direction, pes1, pes2, nod1, nod2, p1, p2, ranu, ranv):
        self.pts1, self.pts2, self.pts3, self.pes1, self.pes2, self.nod1, self.nod2, self.p1, self.p2 = pts1, pts2, pts3, pes1, pes2, nod1, nod2, p1, p2
        self.direction = direction
        self.u = np.linspace(ranu[0], ranu[1]-10**(-10), 50)
        self.v = np.linspace(ranv[0], ranv[1]-10**(-10), 50)
        self.U, self.V = np.meshgrid(self.u, self.v)
        self.objeto = Nurbs_surface(self.pts1, self.pts2, self.pts3, self.direction, self.pes1, self.pes2, self.nod1, self.nod2, self.p1, self.p2)
        self.objeto1 = Nurbs_curbs()
    
    def grafica(self, pdir=True, pgen=True, c_pdir= True, c_pgen=True, enmaller=True, pts_ctrl=True, superficie=True):
        if superficie is True:    #datos para la superficie
            my_surface = self.objeto.procesar(u=self.U, v=self.V)
            X = my_surface[0]
            Y = my_surface[1]
            Z = my_surface[2]

        if enmaller is True:     #datos para el enmallado
            Data = []
            for i in range(len(self.pts1)):
                l_dat = []
                for j in range(len(self.pts2)):
                    l_dat.append(self.objeto.P_ij(i, j))      #data_ctrlpts(i, j, pgen1=pgen, pdir1=pdir)
                Data.append(l_dat)
            Data = np.array(Data)
        
        def funci(lista):
            mi_lista = [[] for k in range(len(lista[0]))]
            for i in range(len(mi_lista)):
                mi_lista[i].append(np.array([k[i] for k in lista]))
            mi_lista = np.concatenate(np.array(mi_lista), axis=0)
            return mi_lista
            
        def funci1(matrix):
            columna = matrix.shape[1]
            lista_alm1 = [[] for k in range(columna)]
            for i in range(len(lista_alm1)):
                lista_alm1[i].append(matrix[: ,i])
            lista_alm1 = np.concatenate(np.array(lista_alm1), axis=0)
            return lista_alm1

        def grafica_maller(matrix):
            mi_lista1 = funci1(matrix)
            mi_lista2 = []
            for i in range(len(mi_lista1)):
                mi_lista2.append(funci(mi_lista1[i]))
            return mi_lista2
        
        if pts_ctrl is True:    #para puntos de control
            mi_puntos = funci(np.concatenate(Data, axis=0))
        
        if pdir is True:    #datos para puntos de curvas directriz y generatriz
            x_pdir = np.array([k[0] for k in self.direction]) #pts1
            y_pdir = np.array([k[1] for k in self.direction])
            z_pdir = np.array([k[2] for k in self.direction])
        if pgen is True:  
            x_pgen = np.array([k[0] for k in self.pts2])
            y_pgen = np.array([k[1] for k in self.pts2])
            z_pgen = np.array([k[2] for k in self.pts2])

        if c_pdir is True:   #para las curvas directriz y generatriz
            C_c1 = self.objeto1.curva1(pts=self.direction, pes=self.pes1, nod=self.nod1, p=self.p1, u=self.u)
            my_c1 = funci(C_c1)
        if c_pgen is True:
            C_c2 = self.objeto1.curva1(pts=self.pts2, pes=self.pes2, nod=self.nod2, p=self.p2, u=self.v)
            my_c2 = funci(C_c2)

        fig = go.Figure()
        if pdir is True:
            fig.add_trace(go.Scatter3d(x=x_pdir, y=y_pdir, z=z_pdir, mode='lines', line=dict(color='black',width=6), showlegend=False))
        if pgen is True:
            fig.add_trace(go.Scatter3d(x=x_pgen, y=y_pgen, z=z_pgen, mode='lines', line=dict(color='black',width=6), showlegend=False))
        if c_pdir is True:
            fig.add_trace(go.Scatter3d(x=my_c1[0], y=my_c1[1], z=my_c1[2], mode='lines', line=dict(color='#FFD700',width=8), showlegend=False))
        if c_pgen is True:
            fig.add_trace(go.Scatter3d(x=my_c2[0], y=my_c2[1], z=my_c2[2], mode='lines', line=dict(color='#FFD700',width=8), showlegend=False))

        
        if enmaller is True:
            lista_graf = grafica_maller(Data)
            lista_graf1 = []
            for i in range(Data.shape[0]):
                lista_graf1.append(funci(Data[i, :]))
            for i in range(len(lista_graf)):
                fig.add_trace(go.Scatter3d(x=lista_graf[i][0], y=lista_graf[i][1], z=lista_graf[i][2], mode='lines', line=dict(color='blue'), showlegend=False))
            for i in range(len(lista_graf1)):
                fig.add_trace(go.Scatter3d(x=lista_graf1[i][0], y=lista_graf1[i][1], z=lista_graf1[i][2], mode='lines', line=dict(color='blue'), showlegend=False))
        if pts_ctrl is True:
            fig.add_trace(go.Scatter3d(x=mi_puntos[0], y=mi_puntos[1], z=mi_puntos[2], mode='markers', marker=dict(color='orange',size=5), showlegend=False))
        if superficie is True:
            fig.add_trace(go.Surface(z=Z, x=X, y=Y, colorscale='viridis', showscale=False, opacity=1,
                                            contours=dict(
                                            x=dict(show=False, color="black"),  # Líneas en X
                                            y=dict(show=False, color="black"),  # Líneas en Y
                                            z=dict(show=False, color="black"),  # Líneas en Z  
                                            )  ))

            #lines = []
            for i in range(X.shape[0]):
                fig.add_trace(go.Scatter3d(x=X[i], y=Y[i], z=Z[i],
                                          mode='lines',
                                          line=dict(color='black', width=1),
                                          showlegend=False))
            for j in range(X.shape[1]):
                fig.add_trace(go.Scatter3d(x=X[:,j], y=Y[:,j], z=Z[:,j],
                                          mode='lines',
                                          line=dict(color='black', width=1),
                                          showlegend=False))

        fig.update_layout(
            scene=dict(
                xaxis=dict(showbackground=False, showgrid=False, showline=False, visible=False),
                yaxis=dict(showbackground=False, showgrid=False, showline=False, visible=False),
                zaxis=dict(showbackground=False, showgrid=False, showline=False, visible=False),
                aspectmode='data',
            ),
            width=800,
            height=800
        )
        return fig 

## Cinta de Mobius
La cinta de Mobius se genera a partir de una recta que empieza a rotar alrededor del eje Z y al mismo tiempo dicha recta gira, de modo que al completar su rotación la recta vuelve a la posición inicial.

In [ ]:
#Puntos de la curva directriz (circunferencia)
rot = Construccion(n=4, r=1).polygon()
rot = [psdr + [0] for psdr in rot]  #esta en el plano xy y hace el papel de pst1

#Puntos de la semicircunferencia
gir =Construccion(n=8, r=1).polygon()[ :len(rot) ]
gir = [psdr[:1] + [0] + psdr[1:] for psdr in gir] #esta en el plano xz y hace el papel de pst3

#Aquí estan los puntos de la recta curva generatriz(P_i).
gen = [[-0.5,0,0],[0.5,0,0]]

# En este caso es la misma curva de pts1
direccion = Construccion(n=4, r=1).polygon()
direccion = [psdr + [0] for psdr in direccion]  #esta en el plano xy y hace el papel de direction

#Pesos de la curva directriz y generatriz reespectivamente
w1 = Construccion(n=4, r=1).peso()
w2 = [1, 1]

#Nodos de la curva directriz y generatriz respectivamente.
x1 = Construccion(n=4, r=1).open1()
x2 = [0,0,1,1]

In [ ]:
mobius = Nurbs_superficie3D_graph(pts1=rot, pts2=gen, pts3=gir, direction=direccion, pes1=w1, pes2=w2,
                                         nod1=x1, nod2=x2, p1=2, p2=1, ranu=[0,4], ranv=[0,1])
mobius.grafica(pdir=False, pgen=False, c_pdir=False, c_pgen=False, enmaller=False, pts_ctrl=False, superficie=True)

# Botella de Klein
Esta superficie se genera con la curva del ocho rotando alrededor del eje Z y al mismo tiempo girando.

In [ ]:
gen= np.array([[0,0,0],[1,0,2],[1,0,0],[1,0,-2],[0,0,0],[-1,0,2],[-1,0,0],[-1,0,-2],[0,0,0]])

rot = Construccion(n=4, r=1).polygon()
rot = [psdr + [0] for psdr in rot]  #esta en el plano xy y hace el papel de pst1

gir =Construccion(n=8, r=1).polygon()[ :len(rot) ]
gir = [psdr[:1] + [0] + psdr[1:] for psdr in gir] #esta en el plano xz y hace el papel de pst3

direccion = Construccion(n=4, r=2).polygon()
direccion = [psdr + [0] for psdr in direccion]  #esta en el plano xy y hace el papel de direction

w1 = Construccion(n=4, r=1).peso()
w2 = [1, 1/2, 1, 1/2, 1, 1/2, 1, 1/2, 1]

x1 = Construccion(n=4, r=1).open1()
x2 = [0,0,0,1,1,2,2,3,3,4,4,4]

In [ ]:
klein = Nurbs_superficie3D_graph(pts1=rot, pts2=gen, pts3=gir, direction=direccion, 
                                          pes1=w1, pes2=w2, nod1=x1, nod2=x2, p1=2, p2=2, ranu=[0,3], ranv=[0,4])
klein.grafica(pdir=False, pgen=False, c_pdir=False, c_pgen=False, enmaller=False, pts_ctrl=False, superficie=True)

# Toro 
En comparación de un toro tradicional que se genera rotando una circunferencia alrededor de un eje, esta superficie también permite que la circunferencia gire al mismo tiempo que rota. Visualmente se aprecie estos movimientos en las líneas que aparecen sobre la superficie.

In [ ]:
gen= Construccion(n=4, r=1).polygon() #esta en el plano xz 
gen = [[psdr[0]]+[0]+[psdr[1]] for psdr in gen]

rot = Construccion(n=4, r=1).polygon()
rot = [psdr + [0] for psdr in rot]  #esta en el plano xy y hace el papel de pst1

gir =Construccion(n=8, r=1).polygon()[ :len(rot) ]
gir = [psdr[:1] + [0] + psdr[1:] for psdr in gir] #esta en el plano xz y hace el papel de pst3

direccion = Construccion(n=4, r=2).polygon()
direccion = [psdr + [0] for psdr in direccion]  #esta en el plano xy y hace el papel de direction

w1 = Construccion(n=4, r=1).peso()
w2 = w1

x1 = Construccion(n=4, r=1).open1()
x2 = x1

In [ ]:
toro = Nurbs_superficie3D_graph(pts1=rot, pts2=gen, pts3=gir, direction=direccion, 
                                          pes1=w1, pes2=w2, nod1=x1, nod2=x2, p1=2, p2=2, ranu=[0,3.6], ranv=[0,4])
toro.grafica(pdir=False, pgen=False, c_pdir=False, c_pgen=False, enmaller=False, pts_ctrl=False, superficie=True)

**Nota:** Esta clase permite controlar que parte de la superficie se desea graficar con los parámetros ranu y ranv. El primero controla si la superficie realiza la rotación completa, mientras que el segundo controla si se grafica la curva generatriz completa. Además cabe resaltar que el rango de estos parámetros dependen de los nodos correspondientes a las curvas directriz y generatriz.